<a href="https://colab.research.google.com/github/adc257/info4940-sitcom/blob/main/sitcom_T5_(with_nContext).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# The Big Bang Theory T5 nContext

# Imports:

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [1]:
!git clone https://github.com/adc257/info4940-sitcom.git

fatal: destination path 'info4940-sitcom' already exists and is not an empty directory.


In [4]:
# !pip install -q condacolab
# import condacolab
# condacolab.install()

⏬ Downloading https://github.com/conda-forge/miniforge/releases/download/23.11.0-0/Mambaforge-23.11.0-0-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:09
🔁 Restarting kernel...


In [2]:
!python -m venv 3350
!source 3350/bin/activate

In [ ]:
# !pip install transformers
# !pip install transformers[torch]
# !pip install accelerate -U
# !pip install accelerate>=0.21.0
# !pip install --upgrade pip
# pip uninstall transformers accelerate
# pip install transformers[torch]
# !pip install --upgrade setuptools

# !pip install accelerate

In [4]:
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta, time
from pathlib import Path

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
import os
from sklearn.metrics import f1_score, classification_report
from sklearn.ensemble import RandomForestClassifier

In [5]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification, TrainingArguments, Trainer
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset
from tqdm import tqdm
from transformers import T5Tokenizer, T5ForConditionalGeneration


# Formatting the Dialogue:

In [6]:
def list_files(start_path):
    file_paths = []
    for root, dirs, files in os.walk(start_path):
        for file in files:
            file_paths.append(os.path.join(root, file))

    file_paths.sort()
    return file_paths

In [7]:
def transformContextTarget(context, target):
  separator = "\n"
  quotation_token = '"'

  formatted_input = ""

  current_scene = context[0]['Scene']

  formatted_input += "Use the following as context:" + separator*2
  formatted_input += "The scene is: " + current_scene + separator

  for entry in context:
    if current_scene != entry['Scene']:
      current_scene = entry["Scene"]
      formatted_input += "The scene changes to: " + current_scene + separator

    recipients = entry['Recipients']
    recipients_str = ""
    if len(recipients) == 0:
      recipients_str += "to themselves"
    elif len(recipients) == 1:
      recipients_str += recipients[0]
    else:
      recipients_str += ", ".join(recipients[:-1]) + " and " + recipients[-1]

    formatted_input += f"{entry['Speaker']} says {quotation_token}{entry['Dialogue']}{quotation_token} to {recipients_str}."
    formatted_input += separator


  formatted_input += separator + "-" *20 + separator*2

  formatted_input += "Using the previous dialogues above as context, classify the following dialogue below as humorous or not humorous:" + separator*2


  entry = target
  if entry['Scene'] != current_scene:
    current_scene = entry["Scene"]
    formatted_input += "The scene changes to: "
  else:
    formatted_input += "The Scene is still: "

  formatted_input += current_scene + separator


  recipients = entry['Recipients']
  recipients_str = ""
  if len(recipients) == 0:
    recipients_str += "to themselves"
  elif len(recipients) == 1:
    recipients_str += recipients[0]
  else:
    recipients_str += ", ".join(recipients[:-1]) + " and " + recipients[-1]

  formatted_input += f"{entry['Speaker']} says {quotation_token}{entry['Dialogue']}{quotation_token} to {recipients_str}."
  formatted_input += separator*2 + "Classify: [humorous] or [not humorous]."

  return formatted_input

# context = [data[0], data[1], data[2]]
# target = data[3]
# transformContextTarget(context, target)

In [8]:
filePath = "/content/info4940-sitcom/cleaned-data/S1"
nSentenceContext = 3

episodePath = list_files(filePath)[0]

with open(episodePath, 'r') as file:
  data = json.load(file)
  data = [info for (_, info) in data.items()]

print(f"Example raw entry:")
display(data[3])

print("\n\nExample formatted entry (3context):\n")
print(transformContextTarget(data[:3], data[3]))

print(f'Actual: {("isHumor" in data[3])}')

Example raw entry:


{'EpisodeID': 'The Big Bang_S0101',
 'Scene': 'A corridor at a sperm bank.',
 'Recipients': ['Sheldon', 'Receptionist'],
 'Speaker': 'Leonard',
 'Dialogue': 'Excuse me.',
 'Dialogue Start Time': '00:00:23:140000',
 'Dialogue End Time': '00:00:25:060000'}



Example formatted entry (3context):

Use the following as context:

The scene is: A corridor at a sperm bank.
Sheldon says "So if a photon is directed through a plane with two slits in it and either slit is observed, it will not go through both slits. If it's unobserved, it will. However, if it's observed after it's left the plane but before it hits its target, it won't have gone through both slits." to Leonard.
Leonard says "Agreed. What's your point?" to Sheldon.
Sheldon says "There's no point, I just think it's a good idea for a T-shirt." to Leonard.

--------------------

Using the previous dialogues above as context, classify the following dialogue below as humorous or not humorous:

The Scene is still: A corridor at a sperm bank.
Leonard says "Excuse me." to Sheldon and Receptionist.

Classify: [humorous] or [not humorous].
Actual: False


# Train Test Split:

In [9]:
def generateDialogueWithContext(filePath, nSentenceContext):
  formatted_inputs = []
  labels = []

  for episodePath in list_files(filePath):

    with open(episodePath, 'r') as file:
      data = json.load(file)
      data = [info for (_, info) in data.items()]


      i = nSentenceContext
      while i < len(data):
        context = [data[j] for j in range(i-nSentenceContext, i)]
        target = data[i]

        formatted_inputs.append(transformContextTarget(context, target))

        if "isHumor" in target:
          labels.append(1)
        else:
          labels.append(0)

        i += 1
  return formatted_inputs, labels

In [10]:
dialogue_list1, label_list1 = generateDialogueWithContext('/content/info4940-sitcom/cleaned-data/S1', 3)
dialogue_list2, label_list2 = generateDialogueWithContext('/content/info4940-sitcom/cleaned-data/S2', 3)
dialogue_list3, label_list3 = generateDialogueWithContext('/content/info4940-sitcom/cleaned-data/S3', 3)
dialogue_list4, label_list4 = generateDialogueWithContext('/content/info4940-sitcom/cleaned-data/S4', 3)
dialogue_list5, label_list5 = generateDialogueWithContext('/content/info4940-sitcom/cleaned-data/S5', 3)

In [11]:
train_texts = dialogue_list1 + dialogue_list2 + dialogue_list3 + dialogue_list4
test_texts = dialogue_list5
train_labels = label_list1 + label_list2 + label_list3 + label_list4
test_labels = label_list5

# T5:

In [13]:
tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-xxl")
model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-xxl", device_map="auto")

/usr/local/lib/python3.10/site-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

...

model-00003-of-00005.safetensors:   0%|          | 0.00/9.96G [00:00<?, ?B/s]

model-00004-of-00005.safetensors:   0%|          | 0.00/10.0G [00:00<?, ?B/s]

model-00005-of-00005.safetensors:   0%|          | 0.00/6.06G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [16]:
# Save the tokenizer and model
# tokenizer.save_pretrained("flan_t5_xxl_tokenizer")
# model.save_pretrained("flan_t5_xxl_model")

In [18]:
## Check whether it saved properly

# directory = "flan_t5_xxl_tokenizer"

# if os.path.exists(directory):
#     print(f"Files in '{directory}': {os.listdir(directory)}")
# else:
#     print(f"The directory '{directory}' does not exist.")

In [ ]:
# Testing model:
input_text = "translate English to German: How old are you?"
input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to("cuda")

outputs = model.generate(input_ids, max_new_tokens=50)
print(tokenizer.decode(outputs[0]))

In [27]:
input_text1 = 'Use the following as context:\n\nThe scene is: A corridor at a sperm bank.\nSheldon says "So if a photon is directed through a plane with two slits in it and either slit is observed, it will not go through both slits. If it\'s unobserved, it will. However, if it\'s observed after it\'s left the plane but before it hits its target, it won\'t have gone through both slits." to Leonard.\nLeonard says "Agreed. What\'s your point?" to Sheldon.\nSheldon says "There\'s no point, I just think it\'s a good idea for a T-shirt." to Leonard.\n\n--------------------\n\nUsing the previous dialogues above as context, classify the following dialogue below as humorous or not humorous:\n\nThe Scene is still: A corridor at a sperm bank.\nLeonard says "Excuse me." to Sheldon and Receptionist.\n\nClassify: [humorous] or [not humorous].'
input_ids1 = tokenizer(input_text1, return_tensors="pt").input_ids.to("cuda")

In [29]:
%time outputs = model.generate(input_ids1, max_new_tokens=5)
print(tokenizer.decode(outputs[0]))

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 7.15 µs
<pad> not humorous</s>


In [30]:
train_texts[0]

'Use the following as context:\n\nThe scene is: A corridor at a sperm bank.\nSheldon says "So if a photon is directed through a plane with two slits in it and either slit is observed, it will not go through both slits. If it\'s unobserved, it will. However, if it\'s observed after it\'s left the plane but before it hits its target, it won\'t have gone through both slits." to Leonard.\nLeonard says "Agreed. What\'s your point?" to Sheldon.\nSheldon says "There\'s no point, I just think it\'s a good idea for a T-shirt." to Leonard.\n\n--------------------\n\nUsing the previous dialogues above as context, classify the following dialogue below as humorous or not humorous:\n\nThe Scene is still: A corridor at a sperm bank.\nLeonard says "Excuse me." to Sheldon and Receptionist.\n\nClassify: [humorous] or [not humorous].'

In [31]:
train_labels[0]

0